## 3.3 数据的索引
《Python数据分析》课程讲义Pandas部分, 参考教材为《Python数据科学手册》

---

第 2 章具体介绍了获取、设置、调整 NumPy 数组数值的方法与工具, 包括取值操作(如 `arr[2, 1]`)、切片操作(如 `arr[:, 1:5]`)、掩码操作(如 `arr[arr > 0]`)、花式索引操作(如 `arr[0, [1, 5]]`), 以及组合操作(如 `arr[:, [1, 5]]`).

下面介绍 Pandas 的 `Series` 和 `DataFrame` 对象相似的数据获取与调整操作, 它与NumPy的操作模式非常相似. 接下来, 将从简单的一维 `Series` 对象开始, 然后再介绍比较复杂的二维 `DataFrame` 对象.

### 3.3.1 Series数据选择方法
`Series`对象与**一维 NumPy数组**和**标准Python字典**在许多方面都一样. 只要记住这两个类比, 就可以更好地理解 `Series`对象的数据索引与选择模式.

# 3.3.1.1 将Series看作字典
和字典一样, `Series` 对象提供了**键值对**的映射:

import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0], index=['a', 'b', 'c', 'd'])
print(data)


In [2]:
print(data['b']) #0.5

0.5


还可以用Python字典的表达式和方法来检测键(索引)和值:

In [3]:
print(data)
print('-----判断data数据中是否有a索引-------')
print('a' in data)
print('-----功能与index属性一样-------')
print(data.keys()) 
print(data.index)
print(list(data.items()))

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64
-----判断data数据中是否有a索引-------
True
-----功能与index属性一样-------
Index(['a', 'b', 'c', 'd'], dtype='object')
Index(['a', 'b', 'c', 'd'], dtype='object')
[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]


`Series` 对象还可以用字典语法修改数据. **通过增加新的键扩展字典一样, 来增加新的索引值扩展 `Series`**:

In [4]:
data['e'] = 1.25
print(data)

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64


`Series` 对象的可变性是一个非常方便的特性: Pandas 在底层已经为可能发生的内存布局和数据复制自动决策, 用户不需要担心这些问题.

#### 3.3.1.2 将Series看作一维数组
`Series` 不仅有类似字典的操作, 而且还具备与 NumPy 数组一样的数组数据索引功能, 包括**简单索引、切片、掩码、花哨的索引**等操作, 具体示例如下所示:

In [5]:
print('-----将显式索引作为切片, 包括c索引-------')
print(data['a':'c'])
print('-----将隐式整数索引作为切片, 不包括2索引-------')
print(data[0:2])
print('-----掩码索引-------')
print(data[(data > 0.3) & (data < 0.8)])
print('-----花式索引-------')
print(data[['a', 'e']])

-----将显式索引作为切片, 包括c索引-------
a    0.25
b    0.50
c    0.75
dtype: float64
-----将隐式整数索引作为切片, 不包括2索引-------
a    0.25
b    0.50
dtype: float64
-----掩码索引-------
b    0.50
c    0.75
dtype: float64
-----花式索引-------
a    0.25
e    1.25
dtype: float64


在以上示例中, **显式和隐式的切片索引方式很容易造成错误**. 
1. 当使用显式索引(即 `data['a':'c']`)作切片时, 结果**包含**最后一个索引；

2. 而当使用隐式索引(即 `data[0:2]`)作切片时, 结果**不包含**最后一个索引.

#### 3.3.1.3 索引器:`loc`、`iloc`和`ix`

**当`index`为整数时, 显式和隐式切片索引容易引起混淆.** 例如, 如果 `Series` 是显式整数索引, 那么
1. `data[1]` 这样的取值操作会使用**显式索引**.
2. `data[1:3]` 这样的切片操作却会使用**隐式索引**.

In [6]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
print(data)
print('-----取值(简单索引)操作是显式索引------')
print(data[1])
print('-----切片操作是隐式索引------')
print(data[1:3])

1    a
3    b
5    c
dtype: object
-----取值(简单索引)操作是显式索引------
a
-----切片操作是隐式索引------
3    b
5    c
dtype: object


> 为解决上述问题， Pandas 提供了一些**索引器**(indexer)属性来作为取值的方法. 它们不是 `Series` 对象的函数方法, 而是暴露切片接口的属性.

1. 索引器是 `loc` 属性, 表示取值和切片都是**显式**的:

In [7]:
print(data)
print(data.loc[1])
print(data.loc[1:3])

1    a
3    b
5    c
dtype: object
a
1    a
3    b
dtype: object


2. 是 `iloc` 属性, 表示取值和切片都是 Python 形式的**隐式**索引:

In [8]:
print(data.iloc[1])
print(data.iloc[1:3])

b
3    b
5    c
dtype: object


3. 是 `ix`属性(**不用学！已经被淘汰**), 它是前两种索引器的混合形式, 在 `Series` 对象中 `ix` 等价于标准的 `[]`(Python 列表)取值方式. `ix` 索引器主要用于 `DataFrame` 对象, 后面将会介绍.

Python 代码的设计原则之一是“**显式优于隐式**”. 使用 `loc 和 iloc 可以让代码更容易维护, 可读性更高. 特别是在处理整数索引的对象时, 强烈推荐使用这两种索引器. 它们既可以让代码阅读和理解起来更容易, 也能避免因误用索引/切片而产生的小 Bug.


### 3.3.2 DataFrame数据选择方法

> `DataFrame` 在有些方面像**二维或结构化数组**, 在有些方面又像一个**共享索引的若干 `Series` 对象构成的字典**. 这两种类比可以帮助更好地掌握这种数据结构的数据选择方法.

#### 3.3.2.1 将DataFrame看作字典
第一种类比是把 `DataFrame` 当作一个由若干 `Series` 对象构成的字典.让用之前的美国五州面积与人口数据来演示:

In [9]:
area = pd.Series({'California': 423967, 'Texas': 695662, 
                  'New York': 141297, 'Florida': 170312, 
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193, 
                 'New York': 19651127, 'Florida': 19552860, 
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop})
print(data)

              area       pop
California  423967  38332521
Texas       695662  26448193
New York    141297  19651127
Florida     170312  19552860
Illinois    149995  12882135


两个 `Series` 分别构成 `DataFrame` 的一列, 可以通过对**列名进行字典形式(dictionary-style)的取值获取数据**:

In [10]:
print(data['area'])
# print(data['California']) # 不能按照 行 进行索引
print(type(data['area']))

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64
<class 'pandas.core.series.Series'>


同样, 也可以用属性形式(attribute-style)选择纯字符串列名的数据:

In [11]:
print('------为了避免和data属性方法冲突，避免使用属性形式选择列--------')
print(data.area)

------为了避免和data属性方法冲突，避免使用属性形式选择列--------
California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64


对同一个对象进行属性形式与字典形式的列数据, 结果是相同的:

In [12]:
print(data.area is data['area'])

True


**注意1**：上述操作并不是通用的. 如果列名不是纯字符串, 或者列名与 `DataFrame` 的**方法同名**, 那么就不能用属性索引. 例如, `DataFrame` 有一个 `pop()` 方法, 如果用 `data.pop` 就不会获取 `'pop'` 列, 而是显示为方法:

In [13]:
print('------data.pop是方法, 为了避免和data属性方法冲突，避免使用属性形式选择列--------')
print(data.pop)
print(data.pop is data['pop'])

------data.pop是方法, 为了避免和data属性方法冲突，避免使用属性形式选择列--------
<bound method DataFrame.pop of               area       pop
California  423967  38332521
Texas       695662  26448193
New York    141297  19651127
Florida     170312  19552860
Illinois    149995  12882135>
False


**注意2**：避免对用属性形式选择的列直接赋值(即可以用 `data['pop'] = z`, 但不要用 `data.pop = z`). 和前面介绍的 `Series` 对象一样, 还可以用字典形式的语法调整对象, 如果要增加一列可以这样做:

In [14]:
data['density'] = data['pop'] / data['area']
print(data)

              area       pop     density
California  423967  38332521   90.413926
Texas       695662  26448193   38.018740
New York    141297  19651127  139.076746
Florida     170312  19552860  114.806121
Illinois    149995  12882135   85.883763


这里演示了两个 `Series` 对象算术运算的简便语法, 将在 3.4 节进行详细介绍.

#### 3.3.2.2 将DataFrame看作二维数组
前面曾提到, 可以把 `DataFrame` 看成是一个增强版的二维数组, 用 `values` 属性按行查看数组数据:

In [15]:
print(data.values)
print(type(data.values))

[[4.23967000e+05 3.83325210e+07 9.04139261e+01]
 [6.95662000e+05 2.64481930e+07 3.80187404e+01]
 [1.41297000e+05 1.96511270e+07 1.39076746e+02]
 [1.70312000e+05 1.95528600e+07 1.14806121e+02]
 [1.49995000e+05 1.28821350e+07 8.58837628e+01]]
<class 'numpy.ndarray'>


理解了这一点, 就可以把许多数组操作方式用在 `DataFrame` 上. 例如, 可以对 `DataFrame` 进行行列转置:

In [16]:
print(data.T)

           California         Texas      New York       Florida      Illinois
area     4.239670e+05  6.956620e+05  1.412970e+05  1.703120e+05  1.499950e+05
pop      3.833252e+07  2.644819e+07  1.965113e+07  1.955286e+07  1.288214e+07
density  9.041393e+01  3.801874e+01  1.390767e+02  1.148061e+02  8.588376e+01


通过字典形式对列进行取值显然会限制把 `DataFrame` 作为 NumPy 数组可以获得的能力, 尤其是当在 `DataFrame` 数组中使用单个行索引获取一行数据时:

In [17]:
print(data.values[0])
# numpy 中, 二维数组data.values , data.values[0] 是对 行进行索引. 

[4.23967000e+05 3.83325210e+07 9.04139261e+01]


而获取一列数据就需要向 `DataFrame` 传递单个列索引:

In [18]:
print(data['area'])

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64


因此, 在进行数组形式的取值时, 就需要用另一种方法——前面介绍过的 Pandas 索引器 `loc`、`iloc` 和 `ix` 了. 通过 `iloc` 索引器, 就可以像对待 NumPy 数组一样索引 Pandas 的底层数组(Python 的隐式索引), `DataFrame` 的行列标签会**自动保留**在结果中:

In [19]:
print(data.iloc[:3, :2])
print(data.loc[:'Illinois', :'pop'])

              area       pop
California  423967  38332521
Texas       695662  26448193
New York    141297  19651127
              area       pop
California  423967  38332521
Texas       695662  26448193
New York    141297  19651127
Florida     170312  19552860
Illinois    149995  12882135


使用 `ix` 索引器可以实现一种混合效果:

In [20]:
print(data.ix[:3, :'pop'])

AttributeError: 'DataFrame' object has no attribute 'ix'

需要注意的是, `ix` 索引器对于整数索引的处理和之前在 `Series` 对象中介绍的一样, 都容易让人混淆. 任何用于处理 NumPy 形式数据的方法都可以用于这些索引器. 例如, 可以在 `loc` 索引器中结合使用掩码与花哨的索引方法:

In [ ]:
print(data.loc[data.density > 100, ['pop', 'density']])

               pop     density
New York  19651127  139.076746
Florida   19552860  114.806121


任何一种取值方法都可以用于调整数据, 这一点和 NumPy 的常用方法是相同的:

In [ ]:
data.iloc[0, 2] = 90
print(data)

              area       pop     density
California  423967  38332521   90.000000
Texas       695662  26448193   38.018740
New York    141297  19651127  139.076746
Florida     170312  19552860  114.806121
Illinois    149995  12882135   85.883763


如果你想熟练使用 Pandas 的数据操作方法, 我建议你花点时间在一个简单的 `DataFrame` 上练习不同的取值方法, 包括查看索引类型、切片、掩码和花哨的索引操作.



#### 3.3.2.3 其他取值方法(特殊的用法)
还有一些取值方法和前面介绍过的方法不太一样. 它们虽然看着有点奇怪, 但是在实践中还是很好用的. 
1. 如果对单个标签**取值就选择列**.
2. 对多个标签用**切片就选择行**.

In [ ]:
print('------单个标签取值就选择列------')
print(data['area'])
print('------多个标签用切片就选择行------')
print(data['Florida':'Illinois'])

------单个标签取值就选择列------
California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64
------多个标签用切片就选择行------
            area       pop
Florida   170312  19552860
Illinois  149995  12882135


切片也可以不用索引值, 而直接用行数来实现:

In [ ]:
print(data[1:3])

            area       pop     density
Texas     695662  26448193   38.018740
New York  141297  19651127  139.076746


与之类似, **掩码操作也可以直接对每一行进行过滤**, 而不需要使用 `loc` 索引器:

In [ ]:
print(data[data.density > 100])

            area       pop     density
New York  141297  19651127  139.076746
Florida   170312  19552860  114.806121


这两种操作方法其实与 NumPy 数组的语法类似, 虽然它们与 Pandas 的操作习惯不太一致, 但是在实践中非常好用.